**Packages to Install**

Be sure to install the following packages before running the notebook: **matplotlib**, **snowflake-snowpark-python**

In [ ]:
/* setup role */
USE ROLE accountadmin;
CREATE ROLE IF NOT EXISTS ai_sql_demo_role;
GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE ai_sql_demo_role WITH GRANT OPTION;
GRANT CREATE DATABASE ON ACCOUNT TO ROLE ai_sql_demo_role WITH GRANT OPTION;
GRANT ROLE ai_sql_demo_role TO ROLE sysadmin;

/* setup database and schemas */
USE ROLE ai_sql_demo_role;
CREATE WAREHOUSE IF NOT EXISTS ai_sql_demo_wh WITH warehouse_size = SMALL;
USE WAREHOUSE ai_sql_demo_wh; 
CREATE DATABASE IF NOT EXISTS ai_sql_demo_db;
CREATE SCHEMA IF NOT EXISTS ai_sql_demo_db.admin;
CREATE STAGE IF NOT EXISTS ai_sql_demo_db.admin.ai_sql_stage DIRECTORY=(ENABLE=true) ENCRYPTION=(TYPE='SNOWFLAKE_SSE');
CREATE SCHEMA IF NOT EXISTS ai_sql_demo_db.impressions;

/* create table where impressions data will be added */
CREATE OR REPLACE TABLE ai_sql_demo_db.impressions.synthetic_impressions (
    Impression_ID NUMBER,
    Customer_ID STRING,
    Event_Timestamp TIMESTAMP_NTZ,
    Image_File_Name STRING,
    Image_Type STRING, 
    Was_Clicked BOOLEAN,
    IP_Address STRING,
    Latitude FLOAT,
    Longitude FLOAT,
    City STRING,
    State STRING
);

In this demo, we will cover:

### AI SQL and Multi-Modal (Unstructured) Data Analysis for Ads
- New FILE data type
- Using Cortex AI SQL functions on image files
- New AI SQL functions like AI_FILTER and AI_AGG

First step let's upload your images to the **ai_sql_demo_db.admin.ai_sql_stage** stage, then use "Run All Below" for the rest of the notebook.

In [ ]:
# Add packages: matplotlib, snowflake-snowpark-python
import streamlit as st
import pandas as pd
import snowbooks_extras

In [ ]:
USE ROLE AI_SQL_DEMO_ROLE;
/* create a table where the images will be added */
CREATE
OR REPLACE TABLE ai_sql_demo_db.impressions.images AS
SELECT
    TO_FILE(FILE_URL) Img_File,
    *
from
    directory (@AI_SQL_DEMO_DB.ADMIN.AI_SQL_STAGE)
WHERE
    RELATIVE_PATH LIKE '%.png'
    AND RELATIVE_PATH NOT LIKE '%instructions%';

In [ ]:
SELECT
    img_file,
    file_url,
    last_modified
FROM ai_sql_demo_db.impressions.images;

In [ ]:
DESCRIBE TABLE ai_sql_demo_db.impressions.images;

## Creating taglines using AI_COMPLETE
Using the **Complete** function, let's create some fresh taglines for our new campaign's advertisement images.

In [ ]:
SELECT
    img_file,
    ai_complete('pixtral-large', prompt('Extract the text from this image and create an advertising tagline. Just return the tagline itself: {0}', img_file)),
    file_url,
    last_modified
FROM ai_sql_demo_db.impressions.images;


## Filtering Ad images with  **AI_FILTER**
Past data has suggested that including coffee makers in ads boosts click-through rate. Let's use **AI_FILTER** to make sure we have ads with coffee makers in our ad campaign.

In [ ]:
SELECT img_file, relative_path, file_url, last_modified FROM ai_sql_demo_db.impressions.images
WHERE AI_FILTER(prompt('does this image {0} have an espresso machine and/or a coffee pot/french press?', img_file))

## Classifying Ad Types with **AI_CLASSIFY**
Now let's prompt **AI_CLASSIFY** with a list of advertisement type categories so it can help classify the ads for us!

In [ ]:
CREATE OR REPLACE TABLE AI_SQL_DEMO_DB.IMPRESSIONS.IMAGES_CLASSIFIED AS
SELECT
    img_file,
    split_part(relative_path, '/', 2) as file_name,
    --    relative_path as file_name,

    parse_json(AI_CLASSIFY(prompt('please help me classify the type of advertisement within this coffee image {0}', img_file), ['long text', 'product shot', 'lifestyle', 'promo offer'])):labels[0]::text as classification,
    
    file_url, 
    last_modified
FROM ai_sql_demo_db.impressions.images;

SELECT * FROM AI_SQL_DEMO_DB.IMPRESSIONS.IMAGES_CLASSIFIED;

## Aggregating Reviews
Using functions like **AI_AGG()** to distill feedback, we can effectively analyze our 'Fuel Your Rise' ad campaign. This deeper understanding will enable us to craft more targeted advertising and enhance customer reach.

In [ ]:
SELECT * FROM AI_SQL_DEMO_DB.IMPRESSIONS.IMPRESSIONS_WITH_REVIEWS;

In [ ]:
SELECT
image_type,

AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'score the sentiment of the feedback on a scale of 1 to 5 where 1 is very negative and 5 is very positive. Just write 1, 2, 3, 4, or 5. Return the average value only') as sentiment,

AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'Analyze the feedback from these reviews and summarize the feedback in one sentence') as summary 
    
FROM AI_SQL_DEMO_DB.IMPRESSIONS.IMPRESSIONS_WITH_REVIEWS
GROUP BY image_type;

## In Summary


### AI SQL unlocks unstructured Data Analysis, all with the familiarity of SQL
- New FILE data type
- Using Cortex AI SQL functions on image files
- New AI SQL functions like AI_FILTER and AI_AGG
